## **EDA Análisis exploratorio de los datos: Regresión Lineal** 

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns

In [95]:
route = './data/HISTORICO_SUERTES.xlsx'
df = pd.read_excel(route)

PermissionError: [Errno 13] Permission denied: './data/HISTORICO_SUERTES.xlsx'

In [ ]:
df_clean = df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21027 entries, 0 to 21026
Data columns (total 85 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Período                    21027 non-null  int64         
 1   Hacienda                   21027 non-null  int64         
 2   Nombre                     21027 non-null  object        
 3   Zona                       21027 non-null  object        
 4   Tenencia                   21026 non-null  float64       
 5   Suerte                     21027 non-null  object        
 6   Suelo                      17269 non-null  object        
 7   Area Neta                  21027 non-null  float64       
 8   Dist Km                    21022 non-null  float64       
 9   Variedad                   21027 non-null  object        
 10  Cod.Estado #               21027 non-null  int64         
 11  Cod.Estado                 21027 non-null  object        
 12  F.Si

In [ ]:
df.head(5)

,Período,Hacienda,Nombre,Zona,Tenencia,Suerte,Suelo,Area Neta,Dist Km,Variedad,...,Humedad Rel Media Ciclo,Oscilacion Temp Med 0-3,Oscilacion Temp Ciclo,Sum Oscilacion Temp Ciclo,Radicion Solar 0-3,Radiacion Solar Ciclo,Precipitacion 0_3,Precipitacion Ciclo,Evaporacion 0-3,Evaporacion Ciclo
0,201701,80493,LA CONCHA,IP02,51.0,002A,CANTARINA,6.00,4.3,CC85-92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201701,81284,UKRANIA INCAUCA,IP05,81.0,039B,NaN,1.45,NaN,CC85-92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201701,80203,EL AMPARO SAA,IP05,31.0,007,CORINTIAS,8.24,23.0,CC01-1228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201701,81380,SAN JUDAS INCAUCA,IP05,82.0,013A,NaN,1.05,66.5,CC01-1940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201701,80298,JAVA,IP06,31.0,025A,GALPON,4.53,17.0,RB73-2223,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
## **Limpieza y preparación de datos**

#### **Data Leakage**
Se identifican variables no relevantes relacionadas con información de las haciendas. 

In [ ]:
DATA_LEAKAGE = ['Hacienda', 'Nombre', 'Zona', 'Codigo Estacion']

#### **Datos Nulos**
Se extraen porcentajes de nulos para identificar features a eliminar, a imputar y si existen similitudes entre las mismas. 

In [ ]:
null_percent = df.isnull().mean() * 100
null_percent.sort_values(ascending=False).head(50)

Sum Oscilacion Temp Ciclo    100.000000
Fert.Nitrogen.               100.000000
Urea 46%                      96.561564
NITRAX-S                      96.442669
MEZ                           95.382128
MicroZinc                     94.535597
Boro Granul.                  93.888810
Sul.Amonio                    93.546393
NITO_XTEND                    82.503448
Vinaza                        80.510772
Humedad Rel Media Ciclo       78.456271
Temp. Media 0-3               78.456271
Oscilacion Temp Med 0-3       78.456271
Temp. Media Ciclo             78.456271
Temp Max Ciclo                78.456271
Humedad Rel Media 0-3         78.456271
Radiacion Solar Ciclo         78.456271
Evaporacion 0-3               78.456271
Precipitacion Ciclo           78.456271
Evaporacion Ciclo             78.456271
Temp Min Ciclo                78.456271
Oscilacion Temp Ciclo         78.456271
Radicion Solar 0-3            78.456271
Precipitacion 0_3             78.456271
%Infest.Diatrea               73.367575


#### **Columnas Vacías**

Eliminamos columnas vacías del Df


In [ ]:
COLUMNS_TO_DROP = ['Sum Oscilacion Temp Ciclo', 'Fert.Nitrogen.']

#### **Chem Features**

Se observan columnas con porcentajes de nulos demasiado altos [99% - 80%]. Parecen columnas relacionadas a productos o químicos que se utilizaron en la plantación, su ausencia podría indicar que el producto no se usó. Se imputarán con 0's. 


In [ ]:
CHEM_FEATURES = ['Urea 46%', 'NITRAX-S', 'MEZ', 'NITO_XTEND', 'Vinaza', 'MicroZinc', 'Boro Granul.', 'Sul.Amonio']

#### **Weather Features**
Se observa un porcentaje en comun con variables relacionadas con condiciones climáticas (78.456271%) por esa razón, se procede a agruparlas en la categoría de weather features

In [ ]:
WEATHER_FEATURES = ['Humedad Rel Media Ciclo', 'Temp. Media 0-3', 'Oscilacion Temp Med 0-3', 'Temp. Media Ciclo', 'Temp Max Ciclo', 'Humedad Rel Media 0-3 ', 'Radiacion Solar Ciclo', 'Evaporacion 0-3', 'Precipitacion Ciclo', 'Evaporacion Ciclo', 'Temp Min Ciclo', 'Oscilacion Temp Ciclo', 'Radicion Solar 0-3', 'Precipitacion 0_3']

In [ ]:
df_filtered_weather = df.dropna(subset=WEATHER_FEATURES, how="any")
df_filtered_weather.head(3)

,Período,Hacienda,Nombre,Zona,Tenencia,Suerte,Suelo,Area Neta,Dist Km,Variedad,...,Humedad Rel Media Ciclo,Oscilacion Temp Med 0-3,Oscilacion Temp Ciclo,Sum Oscilacion Temp Ciclo,Radicion Solar 0-3,Radiacion Solar Ciclo,Precipitacion 0_3,Precipitacion Ciclo,Evaporacion 0-3,Evaporacion Ciclo
13972,202202,80101,PROVIDENCIA,IP01,11.0,012,PALMIRA,15.61,2.2,CC05-430,...,81.756,14.8,22.763,NaN,419.966,422.141,510.8,1523.1,420.8,1744.2
13973,202202,80101,PROVIDENCIA,IP01,11.0,019C,PALMIRA,6.46,2.2,CC09-535,...,81.756,14.8,22.763,NaN,419.966,422.141,510.8,1523.1,420.8,1744.2
13974,202202,80102,LA PAZ,IP01,11.0,050A,ESMERALDA,13.15,1.8,CC05-430,...,81.713,14.8,22.763,NaN,423.232,423.395,468.1,1422.3,421.7,1716.2


In [ ]:
df_filtered_weather.tail(3)

,Período,Hacienda,Nombre,Zona,Tenencia,Suerte,Suelo,Area Neta,Dist Km,Variedad,...,Humedad Rel Media Ciclo,Oscilacion Temp Med 0-3,Oscilacion Temp Ciclo,Sum Oscilacion Temp Ciclo,Radicion Solar 0-3,Radiacion Solar Ciclo,Precipitacion 0_3,Precipitacion Ciclo,Evaporacion 0-3,Evaporacion Ciclo
21024,202407,80102,LA PAZ,IP01,11.0,054E,LA PAZ,1.85,1.8,CC05-430,...,78.704,17.8,30.945,NaN,461.663,444.528,168.7,1041.6,536.4,1867.2
21025,202407,80104,SAMARIA,IP02,11.0,165,PALMIRA,31.42,3.5,CC05-430,...,78.720,17.8,31.040,NaN,467.911,444.839,177.3,1037.8,544.2,1843.9
21026,202407,80124,SANTA LUCIA,IP02,11.0,001,CORINTIAS,2.22,1.1,CC05-430,...,78.687,17.8,30.890,NaN,459.274,444.372,119.8,1041.6,532.5,1881.2


In [93]:
df_filtered_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4530 entries, 13972 to 21026
Data columns (total 85 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Período                    4530 non-null   int64         
 1   Hacienda                   4530 non-null   int64         
 2   Nombre                     4530 non-null   object        
 3   Zona                       4530 non-null   object        
 4   Tenencia                   4530 non-null   float64       
 5   Suerte                     4530 non-null   object        
 6   Suelo                      4328 non-null   object        
 7   Area Neta                  4530 non-null   float64       
 8   Dist Km                    4530 non-null   float64       
 9   Variedad                   4530 non-null   object        
 10  Cod.Estado #               4530 non-null   int64         
 11  Cod.Estado                 4530 non-null   object        
 12  F.Siem

Además, se observa que `WEATHER_FEATURES` tienen el mismo porcentaje de nulos debido a que solo se tiene estos registros de los periodos 202202 hasta 202407 (4530 registros de los 24027 registros que se tenían). 

Se tienen dos opciones, trabajar cone el dataset original y filtrar los features del clima, para trabajar así solo con otras variables, por otra parte, se puede analizar el subset de 4530 junto las variables del clima